<a href="https://colab.research.google.com/github/kavyajeetbora/PSENet.pytorch/blob/master/PSENet_trial_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import numpy as np

import torch
from torch import nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

In [2]:
!pip install pyclipper

     |████████████████████████████████| 133kB 4.9MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import shutil, os
os.chdir('/content')
directory = '/content/cloned-repo'
if os.path.exists(directory):
  shutil.rmtree(directory)

!git clone https://github.com/kavyajeetbora/PSENet.pytorch.git /content/cloned-repo
print("Cloned the repository")
os.chdir('/content/cloned-repo')
!ls

Cloning into '/content/cloned-repo'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (147/147), done.
remote: Total 547 (delta 86), reused 0 (delta 0), pack-reused 400
Receiving objects: 100% (547/547), 12.65 MiB | 19.78 MiB/s, done.
Resolving deltas: 100% (280/280), done.
Cloned the repository
cal_recall  install_dependencies.sh  PSENet.ipynb	     train.py
config.py   LICENSE		     PSENet_predict.ipynb    utils
dataset     models		     PSENet_training.ipynb
eval.py     predict.py		     PSENet_trial_run.ipynb
imgs	    pse			     README.md


In [0]:
## unzipping the files
from zipfile import ZipFile

def unzip_files(file,output_dir):
  with ZipFile(file, 'r') as zipObj:
    # Extract all the contents of zip file in current directory
    zipObj.extractall(output_dir)
  print('Extracted to',output_dir)

def make_directory(directory):
  if os.path.isdir(directory):
    shutil.rmtree(directory)
  
  os.mkdir(directory)
  print('Created a new directory')

training_data_zip = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/Scene Text Detection Dataset/English and Hindi MLT 2019.zip'

In [6]:
# make directories
make_directory('Training Set')

Created a new directory


In [9]:
%%time
unzip_files(training_data_zip,'Training Set')

Extracted to Training Set
CPU times: user 2.28 s, sys: 869 ms, total: 3.15 s
Wall time: 10 s


In [10]:
print(len(os.listdir('Training Set/Images')))

2000


In [11]:
print(len(os.listdir('Training Set/Annotations')))

2000


In [0]:
from dataset.data_utils import *
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset

In [13]:
train_data = MyDataset('Training Set',transform=transforms.ToTensor())
len(train_data)

1938

In [0]:
device = torch.device("cuda:0")

In [15]:
%%time
train_loader = DataLoader(dataset=train_data, batch_size=4, shuffle=True)
img, labels, training_mask = next(iter(train_loader))
print(img.size(), labels.size(), training_mask.size())

torch.Size([4, 3, 640, 640]) torch.Size([4, 6, 640, 640]) torch.Size([4, 640, 640])
CPU times: user 1.59 s, sys: 168 ms, total: 1.76 s
Wall time: 1.14 s


In [0]:
img, labels, training_mask = img.to(device), labels.to(device), training_mask.to(device)

In [17]:
%%time
from models import PSENet
from models.loss import PSELoss
import torch

model = PSENet(backbone='resnet50', pretrained=True, result_num=6, scale=0.5)
model.to(device)
criterion = PSELoss(Lambda=0.7, ratio=3, reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 81.7MB/s]


CPU times: user 644 ms, sys: 482 ms, total: 1.13 s
Wall time: 2.17 s


In [18]:
y1 = model(img)
loss_c, loss_s, loss = criterion(y1, labels, training_mask)

print(y1.size())
print(loss_c, loss_s, loss)

torch.Size([4, 6, 640, 640])
tensor(0.4419, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.5888, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.4860, device='cuda:0', grad_fn=<AddBackward0>)
